# The poset of normalized ideals of a numerical semigroup

We will make use of the package [numericalsgps](https://gap-packages.github.io/numericalsgps/)

In [1]:
LoadPackage("numericalsgps");

true

The function `isaperylistideal` detect if a list of integers `ap` is the ideal of the semigroup `s`. With the help of this function, `I0` computes the set of normalized ideals of a numerical semigroup.

In [5]:
# detects if a given list is the apery list of an ideal of s
isaperylistideal:=function(ap,s)
    local aps, m;
    if not(IsListOfIntegersNS(ap)) then
        return false;
    fi;

    if not(ForAll(ap, x->x>=0)) then    
        return false;
    fi;
    m:=Multiplicity(s);
    if Length(ap)<>m then
        return false;
    fi;

    aps:=AperyList(s);
    return ForAll(Cartesian([1..m],[1..m]), i-> ap[i[1]]+aps[i[2]] >= ap[1+(i[1]+i[2]-2) mod m]);
end;

# computes the set of normalized ideals of s
I0:=function(s)
    local k, c, v, m;
    m:=Multiplicity(s);
    k:=Concatenation([0],KunzCoordinates(s));
    c:=Cartesian(List([1..m],i->[0..k[i]]));
    c:=Filtered(c, k->isaperylistideal(List([1..m],i->k[i]*m+i-1),s));
    v:=List(c, k->List([1..m],i->k[i]*m+i-1)+s);
    return v;
end;

function( ap, s ) ... end

function( s ) ... end

On the set of normalized ideals of a numerical semigroup the relation $\preceq$ is defined as $I\preceq J$ if $I+K=J$ for some normalized ideal $K$

In [13]:
# i \preceq j if i+k=j for some k
leadd:=function(i,j,ids)
    local cand;
    if not(IsSubset(j,i)) then
        return(false);
    fi;
    cand:=Filtered(ids,k->IsSubset(j,k));
    return ForAny(cand, k->j=i+k);
end;

function( i, j, ids ) ... end

In the following example we see that for $S=\langle 4,5,6,7\rangle$, the relations $\subseteq$ and $\preceq$ on the set of normalized ideals of $S$ are not the same. 

In particular, $\{0,2\}+S\subseteq \{0,1,2\}+S$, while $\{0,2\}+S\not\preceq \{0,1,2\}+S$.

In [29]:
s:=NumericalSemigroup(4,5,6,7);;
is:=I0(s);; #compute the set of normalized ideals of s
f:=Filtered(Cartesian(is,is), p->IsSubset(p[2],p[1]) and not(leadd(p[1],p[2],is)));;
List(f,p->List(p,MinimalGenerators));

[ [ [ 0, 2 ], [ 0, 1, 2 ] ] ]

For $S=\langle 3,5\rangle$, both relations are the same.

In [35]:
s:=NumericalSemigroup(3,5);;
is:=I0(s);; #compute the set of normalized ideals of s
ForAll(Cartesian(is,is),p->IsSubset(p[2],p[1])=leadd(p[1],p[2],is));

true

The following auxiliary function computes the set of Kunz coordinates of a normalized ideal

In [38]:
# Kunz coordinates of an ideal wrt multiplicity of the ambient semigroup
kunzcoordinates:=function(i)
    local s,m,ap;
    s:=AmbientNumericalSemigroupOfIdeal(i);
    m:=Multiplicity(s);
    ap:=AperyList(i);
    return List([2..m],i->(ap[i]-i+1)/m);
end;

function( i ) ... end

We can then compute the set of minimal and maximal elements of a set of ideals with respect to this ordering.

In [40]:
# computes minimal elements of a set of ideals
minimals:=function(l,is)
    return Filtered(l,i->Filtered(l,j->leadd(j,i,is))=[i]);
end;

#computes maximal elements of a set of ideals
maximals:=function(l,is)
    return Filtered(l,i->Filtered(l,j->leadd(i,j,is))=[i]);
end;

function( l, is ) ... end

function( l, is ) ... end

Also, we can calculate the set of uppper and lower bounds of a set with respect to $\preceq$.

In [42]:
# upper bounds for a set of ideals li
upperbounds:=function(li,is)
    return Filtered(is,k->ForAll(li,i->leadd(i,k,is)));
end;

# join for a set of ideals li
join:=function(li,is)
    local maxs,ub;
    ub:=upperbounds(li,is);
    maxs:=minimals(ub,is);
    if Length(maxs)<>1 then 
        return fail;
    fi;
    return maxs[1];
end;


function( li, is ) ... end

function( li, is ) ... end

In [44]:
# lower bounds for a set of ideals li
lowerbounds:=function(li,is)
    return Filtered(is,k->ForAll(li,i->leadd(k,i,is)));
end;

# meet for a set of ideals li
meet:=function(li,is)
    local mins,lb;
    lb:=lowerbounds(li,is);
    mins:=maximals(lb,is);
    if Length(mins)<>1 then 
        return fail;
    fi;
    return mins[1];
end;


function( li, is ) ... end

function( li, is ) ... end

With all this, we can check if the set of normalized ideals of a numerical semigroup is a join semilattice (every two ideals have a supremum).

In [45]:
# the same but with I0(s) as argument
isjoinsemilattice:=function(is)
    local cis,ub, minimals,t;

    minimals:=function(l)
        return Filtered(l,i->Filtered(l,j->leadd(j,i,is))=[i]);
    end;

    #is:=I0(s);
    cis:=IteratorOfCartesianProduct(is,is);
    for t in cis do
        ub:=Filtered(is,i->leadd(t[1],i,is) and leadd(t[2],i,is));
        if Length(minimals(ub))>1 then 
            Info(InfoNumSgps,2,"There is no supremum for ", MinimalGenerators(t[1]), " and ", MinimalGenerators(t[2]));
            return false;
        fi;
    od;
    return true;
end;


function( is ) ... end

Or a meet semilattice (every two ideals have an infimum).

In [46]:
# checks if every two ideals has an infimum
ismeetsemilatticei0:=function(s)
    local is,cis,ub, maximals,t;

    maximals:=function(l)
        return Filtered(l,i->Filtered(l,j->leadd(i,j,is))=[i]);
    end;

    is:=I0(s);
    cis:=IteratorOfCartesianProduct(is,is);
    for t in cis do
        ub:=Filtered(is,i->leadd(i,t[1],is) and leadd(i,t[2],is));
        if Length(maximals(ub))>1 then 
            Info(InfoNumSgps,2,"There is no infimum for ", MinimalGenerators(t[1]), " and ", MinimalGenerators(t[2]));
            return false;
        fi;
    od;
    return true;
end;

function( s ) ... end

We are going to check that for $S=\langle 3,7,8\rangle$, the poset $(\mathcal{I}_0(S),\preceq)$ is a lattice. As it has a maximum and a minimum, it suffices to show that it is a join semilattice.

In [52]:
s:=NumericalSemigroup(3,7,8);;
is:=I0(s);;
isjoinsemilattice(is);

true

In this example, $\subseteq$ and $\preceq$ are not the same.

In [54]:
f:=Filtered(Cartesian(is,is), p->IsSubset(p[2],p[1]) and not(leadd(p[1],p[2],is)));;
List(f,p->List(p,MinimalGenerators));

[ [ [ 0, 4 ], [ 0, 1 ] ] ]

The following example shows that for multiplicity four, there are ideals that are not idempotent and are covered by more than one ideal.

In [58]:
s:=NumericalSemigroup(4,7,9,10);;
is:=I0(s);;
i:=[0,4*2+1,4*2+2,3]+s;;
i+i=i;

false

In [62]:
c:=minimals(Difference(upperbounds([i],is),[i]),is);;
List(c,kunzcoordinates);

[ [ 0, 2, 0 ], [ 1, 2, 0 ], [ 2, 1, 0 ] ]

In [63]:
quarksI0:=function(s)
    local is, isnz;
    is:=I0(s);;
    isnz:=Difference(is,[0+s]);;
    return Filtered(isnz,i->Filtered(isnz,j->i<>j and IsSubset(i,j) and leadd(j,i,isnz))=[]);
end;


function( s ) ... end

Next, we see an example of a numerical semigroup that is not irreducible and all of whose quarks have the same depth.

In [66]:
s:=NumericalSemigroup(4, 9, 14, 19);;
qis:=quarksI0(s);;
List(qis,kunzcoordinates);

[ [ 1, 3, 4 ], [ 2, 2, 4 ], [ 2, 3, 3 ] ]

In [68]:
Set(List(qis,kunzcoordinates),Sum);

[ 8 ]

In [67]:
List(Filtered(qis,i->i+i=i),kunzcoordinates);

[ [ 2, 2, 4 ], [ 2, 3, 3 ] ]